<center><div class="alert alert-block alert-info" style=" line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 18px; color:green"> &nbsp; Bengali.AI Speech Recognition : Our very first submission</b><br><br><b style="font-size: 18px; color:green">HELLO EVERYONE!</b><br>
</div></center>

In this Notebook we'll try to submit our first submission by using a publicly available model.

We'll be using this [BanglaASR](https://huggingface.co/bangla-speech-processing/BanglaASR) model. It is actually a finetuned [whisper](https://openai.com/research/whisper) model on rained [Bangla Mozilla Common Voice Dataset](https://arxiv.org/abs/2206.14053). It's reported performance is 4.58% on 7k validation set which seems pretty good to be a starting point. So let's start with that.

I'll be trying to use the other publicly available models one by one and see how they perform. Then we'll get an idea from where we can start the training/finetuning on this huge dataset.

# Imports

In [ ]:
import os
import librosa
import torch
import torchaudio
import numpy as np

from transformers import WhisperTokenizer
from transformers import WhisperProcessor
from transformers import WhisperFeatureExtractor
from transformers import WhisperForConditionalGeneration
from tqdm import tqdm
import pandas as pd
import soundfile as sf
from pydub import AudioSegment

# Load the model

While submitting notebooks, you need to have internet disabled. So you can't download models from huggingface directly. One workaround is to download the models from huggingface, upload them to kaggle as datasets and then use them!

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_path = "/kaggle/input/bangla-speech-processing-banglaasr/BanglaASR"
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_path)
tokenizer = WhisperTokenizer.from_pretrained(model_path)
processor = WhisperProcessor.from_pretrained(model_path)
model = WhisperForConditionalGeneration.from_pretrained(model_path).to(device)

# Demo inference

We'll write an inference function where we'll give the path as input and it will return the transcription!

First we'll need to convert the audio sampling rate to 16k since WhisperFeatureExtractor requires audios to be sampled at 16k.

In [ ]:
#This code is modified from the source code provided here https://huggingface.co/bangla-speech-processing/BanglaASR
def inference_fn(path):
    speech_array, sampling_rate = sf.read(mp3_path)
    speech_array = librosa.resample(np.asarray(speech_array), orig_sr=sampling_rate, target_sr=16000)
    input_features = feature_extractor(speech_array, sampling_rate=16000, return_tensors="pt").input_features
    predicted_ids = model.generate(inputs=input_features.to(device))[0]
    transcription = processor.decode(predicted_ids, skip_special_tokens=True)
    return transcription


# Inference

In [ ]:
mp3_path = "/kaggle/input/chittagong-1/rec_01_audio_0.wav"
print(f"File name :",mp3_path)
AudioSegment.from_file(mp3_path)

File name : /kaggle/input/chittagong-1/rec_01_audio_0.wav


In [ ]:
import os
test_path = "/kaggle/input/chittagong-1"
files = os.listdir(test_path)
ids = []
sentences = []
for file in tqdm(files):
    ids.append(file.split(".")[0])
    mp3_path = os.path.join(test_path,file)
    prediction = inference_fn(mp3_path)

    #sanity check
    if len(prediction)==0:
        prediction = "<>"

    sentences.append(prediction)


  0%|          | 0/1000 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 1000/1000 [1:03:11<00:00,  3.79s/it]


In [ ]:
==================================================================================================================================================================================================================

In [ ]:
df1 = pd.DataFrame({"External_ID":ids,"Contents":sentences})
df1.head()

,External_ID,Contents
0,rec_14_audio_55,জ্ঞোন্স বাজ্য বাজ্য বাজ্য বাজ্য বাজ্য বাজ্য বা...
1,rec_19_audio_24,"সারহাত উত্তর নামকী, তুই হন স্কুল, উত্তর নায়নী..."
2,rec_14_audio_90,"আইহমান, হাই পরাড়টিকে খোঁড়াত প্রেশাদ পরিণে, এ..."
3,rec_05_audio_0,"জনাতকে নার শ্রীলী নওন, আইদের এই আত্নত্বক করে ফ..."
4,rec_14_audio_124,"আড়াড়ুয় কি পরিমাণ পরিশ্রম গড়ন পরে, ব্যাক এক..."


In [ ]:
df1

,External_ID,Contents
0,rec_14_audio_55,জ্ঞোন্স বাজ্য বাজ্য বাজ্য বাজ্য বাজ্য বাজ্য বা...
1,rec_19_audio_24,"সারহাত উত্তর নামকী, তুই হন স্কুল, উত্তর নায়নী..."
2,rec_14_audio_90,"আইহমান, হাই পরাড়টিকে খোঁড়াত প্রেশাদ পরিণে, এ..."
3,rec_05_audio_0,"জনাতকে নার শ্রীলী নওন, আইদের এই আত্নত্বক করে ফ..."
4,rec_14_audio_124,"আড়াড়ুয় কি পরিমাণ পরিশ্রম গড়ন পরে, ব্যাক এক..."
...,...,...
995,rec_04_audio_6,ডাব্য মানোনো ফরে গড়তুন্ত বাড়িগুর দিহে পদমে হ...
996,rec_16_audio_7,শুধু দুই জন এবং বা একটিতে সজঞ্জ। তার বাকি সুল্...
997,rec_19_audio_108,"যোদের আরও পূর্ণারী দিচায়ী, তোই আরো কিছু গঠি প..."
998,rec_14_audio_81,"পরে ডি-আড়াইয়ান বেম্বে, আরাকগ্যান সমস্য আছে ক..."


In [ ]:
df1.to_csv("submission1.csv",index=False)

In [ ]:
==================================================================================================================================================================================================================

In [ ]:
import os
test_path = "/kaggle/input/chittagong-2"
files = os.listdir(test_path)
ids = []
sentences = []
for file in tqdm(files):
    ids.append(file.split(".")[0])
    mp3_path = os.path.join(test_path,file)
    prediction = inference_fn(mp3_path)

    #sanity check
    if len(prediction)==0:
        prediction = "<>"

    sentences.append(prediction)

  0%|          | 0/774 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
  1%|          | 5/774 [00:18<46:34,  3.63s/it]  

KeyboardInterrupt



In [ ]:
df2 = pd.DataFrame({"External_ID":ids,"Contents":sentences})
df2.head()

,External_ID,Contents
0,rec_32_audio_29,আলো-রোজ্জাবেলার একটি রাজ্জার ছিল আইনকে কোতলাত ...
1,rec_26_audio_43,"তাই এজব উদেখি হনো লাইভ নায়, কিছু নায়, সুলেশ।..."
2,rec_40_audio_29,"ইতে বারেজালিতা টিয়ালোটি বারগর্ধে, হবিদের টিয়..."
3,rec_27_audio_3,তেরাতে হাজ্জির বাঙ্গী জানলা।
4,rec_26_audio_19,পজার্থ করে তিনা বাইরো এই রাতে করত আইনার বাকে হ...


In [ ]:
df2

,External_ID,Contents
0,rec_32_audio_29,আলো-রোজ্জাবেলার একটি রাজ্জার ছিল আইনকে কোতলাত ...
1,rec_26_audio_43,"তাই এজব উদেখি হনো লাইভ নায়, কিছু নায়, সুলেশ।..."
2,rec_40_audio_29,"ইতে বারেজালিতা টিয়ালোটি বারগর্ধে, হবিদের টিয়..."
3,rec_27_audio_3,তেরাতে হাজ্জির বাঙ্গী জানলা।
4,rec_26_audio_19,পজার্থ করে তিনা বাইরো এই রাতে করত আইনার বাকে হ...
...,...,...
769,rec_30_audio_40,এছাড়াত বৈশ্বকৃত নিয়ের শাক্তা সুন্যার বাজন্দু...
770,rec_32_audio_31,রুমি আফরান্ড পন্ত নজহারী তর হত নিয়েশে।
771,rec_29_audio_70,সাঁদিদা হত্যাহ এক বছর ফরে ইয়ানা লাভিলে মাবা ফ...
772,rec_29_audio_10,তার ক্রীয়ই তাইজু মেন্ট ফ্যাক্টর আয়টা হয়েছে ...


In [ ]:
df2.to_csv("submission2.csv",index=False)

In [ ]:
==================================================================================================================================================================================================================

In [ ]:
values1 = df1[['External_ID','Contents']]
values2 = df2[['External_ID','Contents']]


dataframes = [values1, values2]

join = pd.concat(dataframes)
join

,External_ID,Contents
0,rec_14_audio_55,জ্ঞোন্স বাজ্য বাজ্য বাজ্য বাজ্য বাজ্য বাজ্য বা...
1,rec_19_audio_24,"সারহাত উত্তর নামকী, তুই হন স্কুল, উত্তর নায়নী..."
2,rec_14_audio_90,"আইহমান, হাই পরাড়টিকে খোঁড়াত প্রেশাদ পরিণে, এ..."
3,rec_05_audio_0,"জনাতকে নার শ্রীলী নওন, আইদের এই আত্নত্বক করে ফ..."
4,rec_14_audio_124,"আড়াড়ুয় কি পরিমাণ পরিশ্রম গড়ন পরে, ব্যাক এক..."
...,...,...
769,rec_30_audio_40,এছাড়াত বৈশ্বকৃত নিয়ের শাক্তা সুন্যার বাজন্দু...
770,rec_32_audio_31,রুমি আফরান্ড পন্ত নজহারী তর হত নিয়েশে।
771,rec_29_audio_70,সাঁদিদা হত্যাহ এক বছর ফরে ইয়ানা লাভিলে মাবা ফ...
772,rec_29_audio_10,তার ক্রীয়ই তাইজু মেন্ট ফ্যাক্টর আয়টা হয়েছে ...


In [ ]:
join.to_csv("3.4 chittagong-banglaWhisper-inferences.csv")

In [ ]:
print('done')

done
